This notebook includes examples of how to find the coefficients of the numerator and the denominator in terms of the $\mathbf{O}$ and $\mathbf{O}^\prime$ matrices for a n-dimensional system with a given input covariance matrix. We show an example for a 4-D system with a diagonal input covariance. Note that we follow indexing starting from 0 here.

In [14]:
import numpy as np
import sympy as sp
import re
import torch
from spectre.spectrum_general.symbolic_spectrum import symbolic

"""Define the dimensionality of the system"""
n = 3

"""Define the variable for which you want the auto-spectrum"""
i = 0

In [15]:
"""Define the matrix J which is n*n and elements like [a00, a01; a10, a11]"""
J = sp.Matrix([[sp.symbols('a_%d%d' % (k, j)) for j in range(n)] for k in range(n)])
print(J)

Matrix([[a_00, a_01, a_02], [a_10, a_11, a_12], [a_20, a_21, a_22]])


Now we define the noise covariance matrices $\mathbf{L}$ and $\mathbf{S}$.

In [16]:
"""Define the noise variables"""
l = [sp.Symbol('l%d' % (j)) for j in range(0, n)] # this is the noise variable
s = [sp.Symbol('s%d' % (j)) for j in range(0, n)] # this is the sigma variable

# We define this matrix to be diagonal
L = sp.diag(*l)

# L[0, 0] = 0
# L[1, 1] = 0
# L[2, 2] = 0
S = sp.diag(*s)

display(L)
display(S)

Matrix([
[l0,  0,  0],
[ 0, l1,  0],
[ 0,  0, l2]])

Matrix([
[s0,  0,  0],
[ 0, s1,  0],
[ 0,  0, s2]])

In [17]:
"""Initialize a symbolic model object"""
model = symbolic(n, L, S)

All the coefficients of the the powers of $\omega$ in the denominator; displayed in ascending order.

In [18]:
coeffs = model.q_all_coeffs()

for j in range(len(coeffs)):
    print('Coefficient of w^{} is: {}'.format(2*j, coeffs[j]))


Coefficient of w^0 is: (Tr(J**2)**3 - 3*Tr(J**2)*Tr(J**4) + 2*Tr(J**6))/6
Coefficient of w^2 is: (Tr(J**2)**2 - Tr(J**4))/2
Coefficient of w^4 is: Tr(J**2)
Coefficient of w^6 is: 1


All the coeffs of $\omega$ in the numerator; displayed in ascending order

In [19]:
num_coeffs = model.p_auto_all_coeffs(i)

for j in range(len(num_coeffs)):
    print('Coefficient of w^{} is: {}'.format(2*j, num_coeffs[j]))

Coefficient of w^0 is: (l0**2*s0**2*(Tr(O_00**2)**2 - Tr(O_00**4)) + l1**2*s1**2*(Tr(O'_10**2)**2 - Tr(O'_10**4) + Tr(O_10**2)**2 - Tr(O_10**4)) + l2**2*s2**2*(Tr(O'_20**2)**2 - Tr(O'_20**4) + Tr(O_20**2)**2 - Tr(O_20**4)))/2
Coefficient of w^2 is: l0**2*s0**2*Tr(O_00**2) + l1**2*s1**2*(-2*Tr(O'_10)*Tr(O_10) + Tr(O'_10**2) + Tr(O_10)**2) + l2**2*s2**2*(-2*Tr(O'_20)*Tr(O_20) + Tr(O'_20**2) + Tr(O_20)**2)
Coefficient of w^4 is: l0**2*s0**2


In [8]:
"""Optionally convert the experssion to latex, alpha is the index"""
alpha = 1
num_coeffs_str = sp.latex(num_coeffs[alpha], mode='equation')
print(num_coeffs_str)


\begin{equation}l_{1}^{2} s_{1}^{2}\end{equation}


In [9]:
"""Processing the string for printing"""
pattern = r"O'(\d{2})"
num_coeffs_str = re.sub(pattern, r"O^\\prime_{\1}", num_coeffs_str)
num_coeffs_str = num_coeffs_str.replace("O", "\mathbf{O}")
num_coeffs_str = num_coeffs_str.replace("s_", "\sigma_")
num_coeffs_str = num_coeffs_str.replace("\operatorname{Tr}", "\mathbf{Tr}")
num_coeffs_str = num_coeffs_str.replace("\left", "")
num_coeffs_str = num_coeffs_str.replace("\\right", "")
print(num_coeffs_str)

\begin{equation}l_{1}^{2} \sigma_{1}^{2}\end{equation}
